# Import Libraries 

In [ ]:
# !pip install ipython-autotime
# %load_ext autotime

In [1]:
import pandas as pd 
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.colors import ListedColormap
from warnings import filterwarnings 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

pd.options.display.max_columns = None
pd.options.display.max_rows = None
# filterwarnings('ignore')


# Import dataset

In [2]:
# aniList = pd.read_csv(io.BytesIO(uploaded['AnimeList.csv']))
url = 'https://raw.githubusercontent.com/Ninelie-Nananai/AnimeListApi/main/AnimeList.csv'
aniList = pd.read_csv(url)
data = aniList.copy()
data2 = aniList.copy()

# Feature นำเข้า  *****


In [ ]:
aniList.head()

In [ ]:
aniList.columns

# Observe data

ตรวจดูข้อมูลว่ามีกี่ column Type อะไรบ้าง ก่อนนำไปเข้า model

In [ ]:
aniList.info()

In [ ]:
aniList.isna().sum()

In [ ]:
(aniList.isna().sum()/aniList.shape[0])

# มีข้อมูลที่เป็น null ไม่ถึง 1%

# EDA

In [ ]:
# score ใน dataset นี้

plt.figure(figsize=(10,10))
sns.distplot(aniList['score']);

คะแนนที่เยอะที่สุดจะอยู่ระหว่าง 5-7 คะแนน
น้อยสุด 1-3 และ 9-10

0 คะแนนมีค่อนข้างเยอะเพราะใน dataset นี้รวม anime ที่ไม่ได้เป็นทางการเข้าไปด้วย (doujin, มังงะจากเว็ป, ทางบ้านเขียนเอง)

In [ ]:
# จำนวนของ source (ที่มา) ทั้งหมดที่มีใน dataset

plt.figure(figsize=(10,10))
plt.xticks(rotation=90)
sns.histplot(aniList, x='source');

### อันดับ 3 Source ที่เยอะที่สุด
1. อันดับ 1 Unknown (เป็นอนิเมะที่ไม่เป็นทางการ ประเภท Doujin ที่ทำขึ้นมาเองไม่ได้ผ่านบริษัท หรือ Indie)
2. อันดับ 2 Original (สร้างโดยบริษัทผลิตอนิเมะเอง ไม่ได้สร้างหรืออ้างอิงจาก Manga หรือ Novel)
3. อันดับ 3 Manga (สร้างจาก Manga)

อันดับทุดท้าย(น้อยสุด) Radio = 9 , และ Digital Manga = 10

## Unknown คืออะไร

In [ ]:
percentUnknown = (aniList['source'].loc[aniList['source'] == 'Original'].count() / aniList['source'].count()) * 100
percentUnknown

### Unknown เมื่อคิดเป็นเปอร์เข็นต์จากทั้งหมด 23.26%

In [ ]:
WhatIsUnknownType = aniList.loc[aniList['type'] == 'Unknown']
WhatIsUnknownType[['title', 'type', 'genre', 'studio', 'producer', 'source']].head(10)

### Unknown เป็นอนิเมะที่ไม่เป็นทางการ เป็น Doujin ที่ทำขึ้นมาเองไม่ได้ผ่านบริษัท เช่น 
* Indie 
* ทางบ้านเขียนขึ้นมาเอง 
* Manga ตามเพจ 
* Manga ตาม Website



### จำนวนของ type ทั้งหมดที่มีใน dataset

In [ ]:
plt.figure(figsize=(10,10))
sns.histplot(aniList, x='type');

### 3 อันดับ Type ที่เยอะที่สุด
1. TV (เพราะอนิเมะในญี่ปุ่นฉายผ่าน TV เป็นหลัก แล้วค่อยนำออกมาขายผ่าน DVD หรือ Blu-ray)
2. OVA (ตอนพิเศษของอนิเมะที่ส่วนมากไม่เกี่ยวกับเนื้อเรื่อง)
3. Movie (ฉายในโรงหนัง)
-
4. Unknown น้อยที่สุด

In [ ]:
dictGenre = {}
tmpGenreList = []

for i in aniList["genre"]:
    # print(str(i))
    x = str(i).split(", ")
    # print(x)
    for n in x:
        # if n not in tmpGenreList:
        tmpGenreList.append(n)


for x in tmpGenreList:
    if x in dictGenre.keys():
        dictGenre[x] = int(dictGenre[x])+1
    else:
        dictGenre[x] = 1
        
dictGenre   

In [ ]:
# newGenre = pd.DataFrame(data=dictGenre.keys(),columns=['genre'])
# newGenre = pd.DataFrame(data=dictGenre.values(),columns=['total_genre'])

newGenre = pd.DataFrame({ 'genre': dictGenre.keys(), 'total_genre': dictGenre.values() })

plt.figure(figsize=(10,10))
sns.barplot(newGenre['total_genre'], newGenre['genre'])
plt.title('Genre')
plt.xlabel('Total')
plt.ylabel('Name')
plt.show()

### 3 อันดับ Genre ที่เยอะที่สุด
1. Comedy
2. Action
3. Fantasy

อันดับสุดท้าย น้อยสุด Yaoi = 39 , และ Yuri = 41

In [ ]:
newGenre.sort_values("total_genre", ascending=True).head(5)

## 10 อันดับอนิเมะ นับจาก score ที่มีใน dataset

In [ ]:
TopTenAnime = aniList.loc[aniList['rank']> 0.0]
TopTenAnime[['rank', 'title', 'score']].sort_values('rank', ascending=True).head(10)

In [ ]:
tmpTopTenAnime=TopTenAnime[['rank', 'title', 'score']].sort_values('rank', ascending=True).head(10)
plt.figure(figsize=(10,5))
sns.barplot(tmpTopTenAnime['score'],tmpTopTenAnime['title'])
plt.title('Top 10 animes')
plt.xlabel('Popularity Score')
plt.show()

### 5 อันดับ อนิเมะที่ได้รับความนิยมมากที่สุด นับจาก score ที่มีใน dataset
1. Fullmetal Alchemist: Brotherhood
2. Kimi no Na wa.
3. Gintama
4. Steins;Gate 0
5. Steins;Gate

* ลอง head ข้อมูลออกมาดู 

In [ ]:
TopTenAnime[['title', 'score','scored_by']].loc[TopTenAnime['title'] == "Steins;Gate"].head()

* เปรียบเทียบ อันดับ 4 กับ อันดับ 5

In [ ]:
TopTenAnime[['title', 'score', 'scored_by']].loc[TopTenAnime['title'] == "Gintama°"].head()

เมื่อลอง head ออกมา จะเห็นว่า Steins;Gate มีคะแนน 9.14 ซึงน้อยกว่า Gintama 0.01 คะแนนแต่ คนที่โหวดให้ Ginnama กับมีน้อยกว่า Steins;Gate

In [ ]:

rating1 = data2[['anime_id','title', 'genre','scored_by','score']]
rating1.drop_duplicates(subset=['anime_id'],inplace=True)
rating1.reset_index(drop=True,inplace=True)

rating1.shape

In [ ]:
# ค่าเฉลี่ยนของคะแนนโดยรวม
C = rating1['score'].mean()

# เรื่องที่มีอัตราการนับคะแนนโดยรวมมากกว่า 85%
m = rating1['scored_by'].quantile(0.85)
print('m: ', m, 'C: ', C)

In [ ]:
def Iighted_rating(x, m=m, C=C):
    v = x['scored_by']
    R = x['score']
    return (v/(v+m) * R) + (m/(m+v) * C)

    # คำนวณ

In [ ]:
# animes having total rate count greater than 85% is considered into lists_animes
lists_animes =rating1.copy().loc[rating1['scored_by'] >= m]

lists_animes['trusted_score'] = lists_animes.apply(Iighted_rating, axis=1)
lists_animes = lists_animes.sort_values('trusted_score', ascending=False)

lists_animes.shape

In [ ]:
# TOP 10 Most popular anime by trusted score
lists_animes[['title', 'scored_by', 'score', 'trusted_score']].head(10)

### อันดับจริงๆ ของอนิเมะแต่ละเรื่องคำนวณจาก คนที่มาโหวดด้วย

In [ ]:
pop=lists_animes[['title', 'scored_by', 'score', 'trusted_score']].head(10)
plt.figure(figsize=(10,5))
sns.barplot(pop['trusted_score'],pop['title'])
plt.title('Anime Recomendation')
plt.xlabel('Popularity Score')
plt.show()


จะเห็นว่าต่างจากก่อนหน้า เมื่อนำจำนวนคนที่มาโหวดด้วย

ก่อน
1. Fullmetal Alchemist: Brotherhood
2. Kimi no Na wa.
3. Gintama
4. Steins;Gate 0
5. Steins;Gate

หลัง
1. Fullmetal Alchemist: Brotherhood
2. Kimi no Na wa.
3. Steins;Gate
4. Hunter x Hunter (2011)
5. Koe no Katachi

# **Content Based Recomendation**

In [ ]:
lists_animes.head()

In [ ]:
def combine_string(row):
  res = [str(r) for r in row]
  return ' | '.join(res)
  # ฟังก์ชัน combine string

In [ ]:
data3=aniList.copy()

cb = ['title', 'genre','type','studio', 'source'] # 

data3['describe'] = data3[cb].apply(combine_string,axis = 1)
#data3['describe'].fillna(' ')

#data3.head()

# สร้าง dataset ใหม่ ชื่อ data3 และเพิ่ม column ใหม่คือ describe ซึ่งเอา genre type episodes มาผสมกัน

# considering the anime content
#data3['describe']=data3['genre'],data3['type'],data3['episodes']
#data3['describe'].fillna(' ')
#data3.head()

data3.drop_duplicates(subset=['title'],inplace=True)
data3.reset_index(drop=True,inplace=True)
data3.shape


## **Feature Engineering**

*   TfidfVectorizer คือ algorithm ที่เลือกใช้ ซึ่งจะช่วยเราจัดการกับข้อความ Eng 
*   จะคืนค่ามาเป็น matrix ซึ่งแทนความใกล้เคียงกันของข้อมูลที่เราเปรียบเทียบ





In [ ]:
tf = CountVectorizer(ngram_range=(1,2), stop_words='english')
tf_matrix = tf.fit_transform(data3['describe'])

test_simil = cosine_similarity(tf_matrix,tf_matrix)

id=data3['anime_id'].values

test_simil = pd.DataFrame(test_simil, index=id, columns=id)

test_simil.columns = data3['title']

test_simil['anime_name'] = data3['title'].values

test_simil.head()

ซึ่งยิ่งค่า Matrix สูงเท่าไหร่ ก็หมายถึงความใกล้เคียงของข้อมูล สูงสุดคือ 1.000000

# **Modeling**

## **Benchmark**

* เปรียบเทียบ (ข้อมูล และ เวลา) ระหว่าง TfidfVectorizer และ CountVectorize
* เปรียบเทียบ เวลาระหว่าง Cosine Similarity กับ Linear Karnel

In [ ]:
# tf_linear = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
# tf_cosine = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
# cv_linear = CountVectorizer(ngram_range=(1,2), stop_words='english')
# cv_cosine = CountVectorizer(ngram_range=(1,2), stop_words='english')

def Racing(driver, co_driver):

    start = time.time()

    if driver == "tf":
        newData = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
    else:
        newData = CountVectorizer(ngram_range=(1,2), stop_words='english')

    matrix = newData.fit_transform(data3['describe'])

    if co_driver == "cosine":
        similarity = cosine_similarity(matrix,matrix)
    else:
        similarity = linear_kernel(matrix,matrix)

    end = time.time()
    
    timeResults = round(end - start ,2)
    return similarity, timeResults

In [ ]:
tf_simil_cosine, tf_consine_time = Racing("tf", "cosine")
tf_simil_linear, tf_linear_time = Racing("tf", 'linear')
cv_simil_cosine, cv_consine_time = Racing("cnt", 'cosine')
cv_simil_linear, cv_linear_time = Racing("cnt", 'linear')

print(
f"""
    TfidfVectorizer Cosine Similarity   Speed {tf_consine_time}  Second
    TfidfVectorizer Linear Karnel       Speed {tf_linear_time}   Second
    CountVectorizer Cosine Similarity   Speed {cv_consine_time}  Second
    CountVectorizer Linear Karnel       Speed {cv_linear_time}   Second
"""
)

### สรุปได้ว่า  Linear Karnel นั้นมีความเร็วมากกว่า Cosine Similarity หากใช้ TfidfVectorizer 

*   TfidfVectorizer เร็วกว่า CountVectorizer 

*   CountVectorizer ในส่วนของความเร็วระหว่าง Cosine และ Linear ต่างกันน้อยมาก





In [ ]:
#cosine_similarity cell
# เตรียมข้อมูลที่คำนวณได้ให้พร้อมสำหรับการแสดงผล

def PrepareDataForDisplay(newData):
    newId = data3['anime_id'].values
    newData = pd.DataFrame(newData, index=newId, columns=newId)
    newData.columns = data3['title']
    newData['anime_name'] = data3['title'].values
    return newData

# id_tf_linear=data3['anime_id'].values
# tf_simil_linear = pd.DataFrame(tf_simil_linear, index=id_tf_linear, columns=id_tf_linear)
# tf_simil_linear.columns = data3['title']
# tf_simil_linear['anime_name'] = data3['title'].values

tf_simil_cosine = PrepareDataForDisplay(tf_simil_cosine)
tf_simil_linear = PrepareDataForDisplay(tf_simil_linear)
cv_simil_cosine = PrepareDataForDisplay(cv_simil_cosine)
cv_simil_linear = PrepareDataForDisplay(cv_simil_linear)

#  คำนวณค่าเฉลี่ยต่างๆจาก algo TfidfVectorizer

In [ ]:
# โชว์รายการที่ใกล้เคียงกับชื่ออนิเมะที่กรอกโดยอิงจาก genre, episode , type โดยคำนวณมาจาก TfidfVectorizer ซึ่งใช้วิธีแบบ linear_karnel

contentList = ['genre','type','source','studio']

def Similarity(name,pData,total):
  index = pData[pData['anime_name']== name]
  index = index.drop('anime_name',axis = 1 ).T
  index.columns = ['similar_anime']

  for i in range(len(contentList)):
    index[[contentList[i].capitalize()]] = data3[[contentList[i]]].values

  index = index.sort_values(by='similar_anime', ascending=False)

  return pd.DataFrame(index.head(total))

Test Variable

TFvec = tf_simil_linear , tf_simil_cosine
CountVec = cv_simil_linear, cv_simil_cosine

### **เปรียบเทียบ ข้อมูล ระหว่าง Tf_vec กับ Cnt_vec ด้วยการใช้ Linear Karnel**


โดยอิงจากความใกล้เคียงกันของข้อมูลประเภท **Title, Genre, Type, Source, Studio**

In [ ]:
searchAnime = 'Log Horizon'

In [ ]:
Similarity(searchAnime, tf_simil_cosine, 5)

In [ ]:
Similarity(searchAnime, tf_simil_linear, 5)

-> TfidfVectorizer เป็นไปตามที่เราต้องการมากกว่า CountVectorizer เพราะชื่อเรื่อง และ Season มีความใกล้เคียงกัน

### **เปรียบเทียบ ข้อมูล ระหว่าง Tf_vec กับ Cnt_vec ด้วยการใช้ Cosine Similarity**

In [ ]:
Similarity(searchAnime, cv_simil_cosine, 5)

In [ ]:
Similarity(searchAnime, cv_simil_linear, 5)

### ในส่วนนี้จะเป็นการลองเอา cosine มาทดสอบบ้างโดยเทียบกับ linear (ด้านบน)
* ในด้านข้อมูลนั้นไม่ต่างกันมากจะต่างกันคือในลำดับที่ 10 เป็นต้นไป
* ในด้านเวลา 
    *   Cosine Similarity Speed :  4.49  Second
    *   Linear Karnel Speed :  4.24  Second

## **สรุป**

*   ใช้  TfidfVectorizer เพราะชื่ออนิเมะที่ค้นหา ค่อนข้างตรงตามที่เราต้องการ
*   ใช้ Linear เพราะเร็วกว่า Cosine 



# **Pipeline**

In [65]:
class PipeLine:
    def __init__(self, data: pd.DataFrame):
        self.aniList = data.copy()
        self.contentList = ['genre','type','source','studio']
        self.targetColumn = ['title', 'genre','type','studio', 'source'] # 

    def Prepare(self):

        def combine_string(row):
            res = [str(r) for r in row]
            return ' | '.join(res)

        self.aniList['describe'] = self.aniList[self.targetColumn].apply(combine_string,axis = 1)

        self.aniList.drop_duplicates(subset=['title'],inplace=True)
        self.aniList.reset_index(drop=True,inplace=True)

        return self.aniList

    #Feature Engineering
    def Matrix(self, driver, co_driver):
        if driver == "tf":
            newData = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
        else:
            newData = CountVectorizer(ngram_range=(1,2), stop_words='english')

        matrix = newData.fit_transform(self.aniList['describe'])

        if co_driver == "cosine":
            similarity = cosine_similarity(matrix,matrix)
        else:
            similarity = linear_kernel(matrix,matrix)

        return similarity

    # Matrix
    def NewData(self, newData):
        newId = self.aniList['anime_id'].values
        newData = pd.DataFrame(newData, index=newId, columns=newId)
        newData.columns = self.aniList['title']
        newData['anime_name'] = self.aniList['title'].values
        return newData

    def Similarity(self, name,pData):
        index = pData[pData['anime_name'] == name]
        index = index.drop('anime_name',axis = 1 ).T
        index.columns = ['similar_anime']

        for i in range(len(self.contentList)):
            index[[self.contentList[i].capitalize()]] = self.aniList[[self.contentList[i]]].values

        index = index.sort_values(by='similar_anime', ascending=False)

        return pd.DataFrame(index)
        
    # Execute
    def Execution(self, searchAnime):
        self.aniList = self.Prepare()
        tf_simil_linear = self.Matrix("tf", 'linear')
        tf_simil_linear = self.NewData(tf_simil_linear)
        
        return self.Similarity(searchAnime, tf_simil_linear)

In [66]:
pipe = PipeLine(aniList)

In [69]:
result = pipe.Execution("One Piece")

In [70]:
result.head(10)

,similar_anime,Genre,Type,Source,Studio
title,,,,,
One Piece,1.000000,"Action, Adventure, Comedy, Super Power, Drama,...",TV,Manga,Toei Animation
One Piece: Episode of Sorajima,0.563116,"Action, Adventure, Comedy, Super Power, Drama,...",Special,Manga,Toei Animation
One Piece Movie 1,0.550795,"Action, Adventure, Comedy, Super Power, Fantas...",Movie,Manga,Toei Animation
One Piece Film: Z,0.464890,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",Movie,Manga,Toei Animation
One Piece Recap,0.463263,"Action, Adventure, Comedy, Fantasy, Shounen, S...",OVA,Manga,Toei Animation
Toriko,0.457907,"Action, Adventure, Comedy, Fantasy, Shounen",TV,Manga,Toei Animation
One Piece: Adventure of Nebulandia,0.453302,"Action, Adventure, Comedy, Super Power, Fantas...",Special,Manga,Toei Animation
One Piece: Long Ring Long Land-hen,0.442446,"Action, Adventure, Comedy, Super Power, Drama,...",Special,Manga,Toei Animation
One Piece: Glorious Island,0.434724,"Action, Adventure, Comedy, Super Power, Fantas...",Special,Manga,Toei Animation


# **Deploy**

working ...

# **Reference**

* https://www.kaggle.com/lavanyaanandm/recommending-anime-s-using-all-recommendation-sys#10.-Reference-
* https://www.kaggle.com/azathoth42/myanimelist?select=AnimeList.csv
